# IS4226 Mid Term

1. Develop backtesters for these strategies and test the strategy on in-sample data
    1. To keep things simple, all groups will test their strategy on an equal weighted portfolio of MAANG stocks from 01/01/2015 to 31/12/2019 on daily frequency data.
    2. Please note that this time, you need to do just in-sample testing and report what different variations you have tried.  In the final semester project, You will do out-of-sample testing to finalize strategies.
1. Prepare a notebook with significant markups, and please explain your code through comments. 
2. Please also describe the idea/strategy in the notebook at the start. The more understandable your notebook, the better it will be.
3. Report Strategy Risk, Returns, and performance metrics for MAANG stocks portfolio. Visualizations are optional but will be a bonus if you do that.

# Strategy 1 : RSI Momentum

# Introduction

## RSI
[Insert Explanation Here]

## Market Cap Weighted RSI
[Insert Explanation Here]

## Strategy
[Insert Explanation Here]

# Data

In [ ]:
# import yfinance as yf

# msft = yf.Ticker("MSFT")

# # get all stock info
# msft.info

# # get historical market data
# hist = msft.history(period="1mo")

# # show meta information about the history (requires history() to be called first)
# msft.history_metadata

# # show actions (dividends, splits, capital gains)
# msft.actions
# msft.dividends
# msft.splits
# msft.capital_gains  # only for mutual funds & etfs

# # show share count
# msft.get_shares_full(start="2022-01-01", end=None)

# # show financials:
# msft.calendar
# msft.sec_filings
# # - income statement
# msft.income_stmt
# msft.quarterly_income_stmt
# # - balance sheet
# msft.balance_sheet
# msft.quarterly_balance_sheet
# # - cash flow statement
# msft.cashflow
# msft.quarterly_cashflow
# # see `Ticker.get_income_stmt()` for more options

# # show holders
# msft.major_holders
# msft.institutional_holders
# msft.mutualfund_holders
# msft.insider_transactions
# msft.insider_purchases
# msft.insider_roster_holders

# msft.sustainability

# # show recommendations
# msft.recommendations
# msft.recommendations_summary
# msft.upgrades_downgrades

# # show analysts data
# msft.analyst_price_targets
# msft.earnings_estimate
# msft.revenue_estimate
# msft.earnings_history
# msft.eps_trend
# msft.eps_revisions
# msft.growth_estimates

# # Show future and historic earnings dates, returns at most next 4 quarters and last 8 quarters by default.
# # Note: If more are needed use msft.get_earnings_dates(limit=XX) with increased limit argument.
# msft.earnings_dates

# # show ISIN code - *experimental*
# # ISIN = International Securities Identification Number
# msft.isin

# # show options expirations
# msft.options

# # show news
# msft.news

# # get option chain for specific expiration
# opt = msft.option_chain('YYYY-MM-DD')
# # data available via: opt.calls, opt.puts

# tech = yf.Sector('technology')
# software = yf.Industry('software-infrastructure')

# # Common information
# tech.key
# tech.name
# tech.symbol
# tech.ticker
# tech.overview
# tech.top_companies
# tech.research_reports

# # Sector information
# tech.top_etfs
# tech.top_mutual_funds
# tech.industries

# # Industry information
# software.sector_key
# software.sector_name
# software.top_performing_companies
# software.top_growth_companies


In [2]:
import yfinance as yf

yf.Ticker(ticker='META').info

{'address1': '1 Meta Way',
 'city': 'Menlo Park',
 'state': 'CA',
 'zip': '94025',
 'country': 'United States',
 'phone': '650 543 4800',
 'website': 'https://investor.fb.com',
 'industry': 'Internet Content & Information',
 'industryKey': 'internet-content-information',
 'industryDisp': 'Internet Content & Information',
 'sector': 'Communication Services',
 'sectorKey': 'communication-services',
 'sectorDisp': 'Communication Services',
 'longBusinessSummary': 'Meta Platforms, Inc. engages in the development of products that enable people to connect and share with friends and family through mobile devices, personal computers, virtual reality headsets, and wearables worldwide. It operates in two segments, Family of Apps and Reality Labs. The Family of Apps segment offers Facebook, which enables people to share, discuss, discover, and connect with interests; Instagram, a community for sharing photos, videos, and private messages, as well as feed, stories, reels, video, live, and shops; M

## Data Extraction

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict
import yfinance as yf

class YFinanceStrategy:
    def __init__(self, symbol: str, start_date: str, end_date: str, interval: str) -> None:
        self.symbol = symbol
        self.start_date = start_date
        self.end_date = end_date
        self.interval = interval
        self.data = None
        self.trades = None
        self.performance = None

    def prepare_data(self):
        """
        Prepare stock data including market cap for a given symbol. Override if more data is needed.
        
        :param symbol: Stock symbol (e.g., 'AAPL')
        :param start_date: Start date for historical data (e.g., '2020-01-01'). (YYYY-MM-DD)
        :param end_date: End date for historical data(e.g., '2020-01-01'). (YYYY-MM-DD)
        :param interval: Data interval (default is '1d' for daily)
        :return: DataFrame with prepared stock data
        """
        # Fetch stock data
        data = yf.Ticker(self.symbol)
        
        # Get historical data
        df = data.history(start=self.start_date, end=self.end_date, interval=self.interval).reset_index()
        
        # Convert date to UTC timestamp
        df['timestamp_utc'] = (pd.to_datetime(df['Date']).dt.tz_convert('UTC').astype(int) // 10**6)
        
        # Calculate returns and log returns
        df['returns_close'] = (df['Close'] / df['Close'].shift(1)) - 1
        df['logreturns_close'] = np.log(df['Close'] / df['Close'].shift(1))
        
        # Fetch market cap
        info = data.info
        market_cap = info.get('marketCap', None)
        
        # Add market cap as a new column
        # Note: This will be the same value for all rows, representing the current market cap
        df['MC'] = market_cap
        
        # Sort and reset index
        df = df.sort_values(by='timestamp_utc', ascending=True).reset_index(drop=True)
        
        return df


    def strategy(self) -> pd.DataFrame:
        """
        Generate a signal column, 1 for Long, -1 for Short, 0 for no position.
        """
        raise NotImplementedError("Strategy method must be implemented")

    def calculate_trades(self) -> pd.DataFrame:
        """
        Calculate trades based on the strategy signal.
        """
        if self.data is None:
            raise ValueError("Data must be fetched first. Try calling self.fetch_data()")

    def calculate_strategy_performance(self):
        """
        Calculate strategy performance metrics. calculate_trades() must be called first.
        """
        if self.trades is None:
            raise ValueError("Trades must be calculated first. Try calling self.calculate_trades()")

    def plot_performance(self) -> None:
        """
        Plot strategy performance metrics
        """
        if self.performance is None:
            raise ValueError("Performance metrics must be calculated first. Try calling self.calculate_strategy_performance()")
    """
    Getters
    """
    def get_trades(self) -> pd.DataFrame:
        return self.trades

    def get_data(self) -> pd.DataFrame:
        return self.data

    def get_performance(self) -> Dict[str, float]:
        return self.performance


class YFinancePortfolio:
    def __init__(self, strategies: List[YFinanceStrategy], weights: List[float] = None, benchmark_symbol : str = "^GSPC") -> None:
        self.strategies = strategies
        self.weights = weights if weights is not None else self._equal_weights()
        self.portfolio_performance = None

    def _equal_weights(self) -> List[float]:
        return [1.0 / len(self.strategies)] * len(self.strategies)
    def set_weights(self, weights: List[float]) -> None:
        if len(weights) != len(self.strategies):
            raise ValueError("Number of weights must match number of strategies")
        if not np.isclose(sum(weights), 1.0):
            raise ValueError("Weights must sum to 1.0")
        self.weights = weights

    def calculate_portfolio_performance(self) -> Dict[str, float]:
        # Implement overall portfolio performance calculation
        pass

    def plot_portfolio_performance(self) -> None:
        # Implement portfolio performance plotting
        pass

    def get_portfolio_performance(self) -> Dict[str, float]:
        return self.portfolio_performance

## Data Analysis

# Strategy Development

# Optimization and Model Selection
[Insert Explanation Here]

# Performance Analysis

## Win Rate / Returns
[Insert Explanation Here]


## Risk
[Insert Explanation Here]


## Performance Metrics
[Insert Explanation Here]


# Limitations
[Insert Explanation Here]



# Conclusion
[Insert Explanation Here]

In [ ]:
top_25_tech_stocks = [
    "AAPL",  # Apple
    "MSFT",  # Microsoft
    "GOOGL",  # Alphabet (Google)
    "AMZN",  # Amazon
    "NVDA",  # NVIDIA
    "META",  # Meta Platforms
    "TSLA",  # Tesla
    "TSM",   # Taiwan Semiconductor Manufacturing Company
    "AVGO",  # Broadcom
    "ORCL",  # Oracle
    "CSCO",  # Cisco Systems
    "ADBE",  # Adobe
    "CRM",   # Salesforce
    "ASML",  # ASML Holding
    "AMD",   # Advanced Micro Devices
    "ACN",   # Accenture
    "INTC",  # Intel
    "QCOM",  # Qualcomm
    "IBM",   # IBM
    "TXN",   # Texas Instruments
    "PYPL",  # PayPal
    "INTU",  # Intuit
    "AMAT",  # Applied Materials
    "NOW",   # ServiceNow
    "ADP"    # Automatic Data Processing
]

